In [ ]:
include("../analysis/mESC/load_analysis.jl")
include("../analysis/mESC/filter_prior.jl")

srcpath = normpath(srcpath*"../age-dependent/")
fitpath = datapath*"fits_age-dependent/"
include(srcpath*"dists.jl")
include(srcpath*"mle.jl")

# fix the replication point to be in the middle of the S phase
θᵣ = θ_G1_S + (θ_S_G2M - θ_G1_S)/2

fits_main = load(fitpath*"fits_main.jld2", "fits_main")
include("../analysis/mESC/filter_post.jl");

# Fig. 4A: Cell division model vs age-independent model fits (5 example genes)

Note that for the $\theta$-independent model we infer the burst frequency normalised by the degradation rate, $f/d$. For a proper comparison between the age-dependent and age-independent models we need to consider the absolute parameter values.

In [62]:
burst_freqs_G1_th_ind = get_burst_frequency.(G1_th_ind_fits) .* decay_rates
burst_freqs_G2M_th_ind = get_burst_frequency.(G2M_th_ind_fits) .* decay_rates
burst_sizes_G1_th_ind = get_burst_size.(G1_th_ind_fits)
burst_sizes_G2M_th_ind = get_burst_size.(G2M_th_ind_fits);

In [63]:
burst_freqs_G1_th_dep = get_burst_frequency_G1.(fits_main)
burst_freqs_G2M_th_dep = get_burst_frequency_G2M.(fits_main)
burst_sizes_G1_th_dep = get_burst_size_G1.(fits_main, Ref(thetaG1))
burst_sizes_G2M_th_dep = get_burst_size_G2M.(fits_main, Ref(thetaG2M));

In [64]:
ratio_f_G1 = burst_freqs_G1_th_dep ./ burst_freqs_G1_th_ind
ratio_f_G2M = burst_freqs_G2M_th_dep ./ burst_freqs_G2M_th_ind
ratio_b_G1 = burst_sizes_G1_th_dep ./ burst_sizes_G1_th_ind
ratio_b_G2M = burst_sizes_G2M_th_dep ./ burst_sizes_G2M_th_ind;

In [ ]:
genes = ["Klf4", "Fxyd6", "Acin1", "Lrpap1", "Fbxo5"]
@show genes
inds = [findfirst(gene_names .== gene) for gene in genes];

In [ ]:
include(srcpath*"confidence_main.jl")

In [ ]:
CI_ratio_f_G1 = get_ratio_confidence_intervals(G1_th_ind_fits[inds], 2, fits_main[inds], counts_spliced[inds], theta,
                                               "G1", T_cycle, decay_rates[inds], θᵣ, θ_G1_S, θ_S_G2M, get_burst_frequency_ratio_CI_PL,
                                               alpha=0.05, resolution=100, verbose=true, time_limit=30.0, error_check=false)

In [ ]:
GC.gc()
CI_ratio_b_G1 = get_ratio_confidence_intervals(G1_th_ind_fits[inds], 2, fits_main[inds], counts_spliced[inds], theta,
                                               "G1", T_cycle, decay_rates[inds], θᵣ, θ_G1_S, θ_S_G2M, get_burst_size_ratio_CI_PL,
                                               alpha=0.05, resolution=100, verbose=false, time_limit=30.0, error_check=false)

In [ ]:
GC.gc()
CI_ratio_f_G2M = get_ratio_confidence_intervals(G2M_th_ind_fits[inds], 4, fits_main[inds], counts_spliced[inds], theta,
                                                "G2/M", T_cycle, decay_rates[inds], θᵣ, θ_G1_S, θ_S_G2M, get_burst_frequency_ratio_CI_PL,
                                                alpha=0.05, resolution=100, verbose=false, time_limit=30.0, error_check=false)

In [ ]:
GC.gc()
CI_ratio_b_G2M = get_ratio_confidence_intervals(G2M_th_ind_fits[inds], 4, fits_main[inds], counts_spliced[inds], theta,
                                                "G2/M", T_cycle, decay_rates[inds], θᵣ, θ_G1_S, θ_S_G2M, get_burst_size_ratio_CI_PL,
                                                alpha=0.05, resolution=100, verbose=false, time_limit=30.0, error_check=false)

In [ ]:
f = Figure(size = (size_pt[1]*0.9, size_pt[2]*1.5), figure_padding = 1)
ga = f[1,1] = GridLayout()

ax = Axis(ga[1,1],
          xlabel="", ylabel="Burst frequency ratio", xticklabelsvisible=false)
x = 1:length(genes)
ax.xticks = (x, genes)

y1 = ratio_f_G1[inds]
y2 = ratio_f_G2M[inds]
y1_lb = first.(CI_ratio_f_G1)
y2_lb = first.(CI_ratio_f_G2M)
y1_ub = last.(CI_ratio_f_G1)
y2_ub = last.(CI_ratio_f_G2M)
hlines!(ax, 1, color=(:black, 0.5), linestyle=:dash)
scatter!(ax, x, y1, markersize=7, color=(c1, 0.95))
scatter!(ax, x, y2, markersize=7, color=(c2, 0.95))
errorbars!(ax, x, y1, y1 .- y1_lb, y1_ub .- y1, whiskerwidth=5, color=(c1, 0.95))
errorbars!(ax, x, y2, y2 .- y2_lb, y2_ub .- y2, whiskerwidth=5, color=(c2, 0.95))
ylims!(ax, (nothing, 3.5))

ax = Axis(ga[2,1],
          xlabel="", ylabel="Burst size ratio")
x = 1:length(genes)
ax.xticks = (x, genes)

y1 = ratio_b_G1[inds]
y2 = ratio_b_G2M[inds]
y1_lb = first.(CI_ratio_b_G1)
y2_lb = first.(CI_ratio_b_G2M)
y1_ub = last.(CI_ratio_b_G1)
y2_ub = last.(CI_ratio_b_G2M)
hlines!(ax, 1, color=(:black, 0.5), linestyle=:dash)

scatter!(ax, x, y1, markersize=7, color=(c1, 0.95))
scatter!(ax, x, y2, markersize=7, color=(c2, 0.95))
errorbars!(ax, x, y1, y1 .- y1_lb, y1_ub .- y1, whiskerwidth=5, color=(c1, 0.95))
errorbars!(ax, x, y2, y2 .- y2_lb, y2_ub .- y2, whiskerwidth=5, color=(c2, 0.95))

rowgap!(ga, 3)
f

# Fig. 4B: Cell division model vs age-independent model fits (all genes)

In [72]:
f = Figure(size = (size_pt[1]*0.9, size_pt[2]*1.7), figure_padding = 1)
ga = f[1,1] = GridLayout()

y1 = ratio_f_G1; x1 = fill("G1", length(y1))
y2 = ratio_f_G2M; x2 = fill("G2/M", length(y2))
ax1 = Axis(ga[1,1], xlabel="Burst frequency ratio", ylabel="", yticks=(1:2, ["G1", "G2/M"]))
rainclouds!(ax1, vcat(x1, x2), vcat(y1, y2), gap=-0.7,
            orientation = :horizontal,
            color = vcat(fill(c1, length(y1)), fill(c2, length(y2))),
            cloud_width=0.9, show_median=false, violin_limits=(-Inf, Inf),
            boxplot_width=0.1, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
            jitter_width=0.2, markersize=1.3, side_nudge=0.12)
vlines!(1, color=(:black, 0.4), linestyle=:dash)
xlims!(0.0, 5.0)

y1 = ratio_b_G1; x1 = fill("G1", length(y1))
y2 = ratio_b_G2M; x2 = fill("G2/M", length(y2))
ax2 = Axis(ga[2,1], xlabel="Burst size ratio", ylabel="", yticks=(1:2, ["G1", "G2/M"]), xticks=(0:0.5:2.0, ["0", "0.5", "1.0", "1.5", "2"]))
rainclouds!(ax2, vcat(x1, x2), vcat(y1, y2), gap=-0.7,
            orientation = :horizontal,
            color = vcat(fill(c1, length(y1)), fill(c2, length(y2))),
            cloud_width=0.9, show_median=false, violin_limits=(-Inf, Inf),
            boxplot_width=0.1, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
            jitter_width=0.2, markersize=1.3, side_nudge=0.12)
vlines!(1, color=(:black, 0.4), linestyle=:dash)
xlims!(0.0, 2.0)

rowgap!(ga, 7)

In [ ]:
f

# Fig 4C: G2/M vs G1

In [74]:
ratio_f_th_dep = burst_freqs_G2M_th_dep ./ burst_freqs_G1_th_dep
ratio_b_th_dep = burst_sizes_G2M_th_dep ./ burst_sizes_G1_th_dep
ratio_f_th_ind = burst_freqs_G2M_th_ind ./ burst_freqs_G1_th_ind
ratio_b_th_ind = burst_sizes_G2M_th_ind ./ burst_sizes_G1_th_ind;

In [ ]:
@show median(ratio_f_th_dep)
@show quantile(ratio_f_th_dep, 0.25)
@show quantile(ratio_f_th_dep, 0.75);

In [ ]:
@show median(ratio_f_th_ind)
@show quantile(ratio_f_th_ind, 0.25)
@show quantile(ratio_f_th_ind, 0.75);

In [ ]:
@show median(ratio_b_th_dep)
@show quantile(ratio_b_th_dep, 0.25)
@show quantile(ratio_b_th_dep, 0.75);

In [ ]:
@show median(ratio_b_th_ind)
@show quantile(ratio_b_th_ind, 0.25)
@show quantile(ratio_b_th_ind, 0.75);

In [79]:
x = ratio_f_th_dep
y = ratio_b_th_dep
f = Figure(size = (size_pt[1]*1.1, size_pt[2]*1.2), figure_padding = 1)

ga = f[1, 1] = GridLayout()
axtop = Axis(ga[1, 1], 
             leftspinevisible = false,
             rightspinevisible = false,
             bottomspinevisible = false,
             topspinevisible = false)
axmain = Axis(ga[2, 1], xlabel = "", ylabel = "",
              yminorticks = IntervalsBetween(2),
              yminorticksvisible = true,
              yminorticksize = 1.5,
              yminortickwidth = 0.7,
              xticksmirrored = true,
              yticksmirrored = true,
              rightspinecolor = (c1, 1),
              topspinecolor = (c2, 1))
axright = Axis(ga[2, 2],
               leftspinevisible = false,
               rightspinevisible = false,
               bottomspinevisible = false,
               topspinevisible = false)

linkyaxes!(axmain, axright)
linkxaxes!(axmain, axtop)

hidedecorations!(axtop, grid = false)
hidedecorations!(axright, grid = false)
scatter!(axmain, x, y, color=(:gray, 0.4), markersize=2)
vlines!(axmain, 1, color=(:black, 0.4), linestyle=:dash)
hlines!(axmain, 1, color=(:black, 0.4), linestyle=:dash)
xlims!(axmain, low = 0, high = 2.0)

density!(axtop, x, color=(c2), npoints=1000)
density!(axtop, ratio_f_th_ind, color=(c2, 0.2), npoints=1000)
hlines!(axtop, 0, color=(:black, 0.3), linewidth=0.3)
boxplot!(axtop, fill(0.0, length(x)), x, orientation=:horizontal, strokewidth = 0.7, 
         width=0.7, whiskerwidth=0, show_outliers=false, color=(c2, 0))
ylims!(axtop, low=-0.4, high=5.7)

density!(axright, y, direction = :y, color=(c1), npoints=1000)
density!(axright, ratio_b_th_ind, direction = :y, color=(c1, 0.2), npoints=1000)
vlines!(axright, 0, color=(:black, 0.3), linewidth=0.3)
boxplot!(axright, fill(0.0, length(y)), y, strokewidth = 0.7, 
         width=0.5, whiskerwidth=0, show_outliers=false, color=(c1, 0))
ylims!(axright, low=0, high=3.0)
xlims!(axright, low=-0.4, high=5.7)

colgap!(ga, 2)
rowgap!(ga, 2)
colsize!(ga, 2, Relative(1.2/3))
rowsize!(ga, 1, Relative(1.2/3))

In [ ]:
f

In [ ]:
cor(ratio_f_th_dep, ratio_b_th_dep)

In [ ]:
cor(ratio_f_th_ind, ratio_b_th_ind)

# Fig 4D: binomial downsampling test

In [83]:
@load datapath*"vec_downsampling_ratios.jld2" inds vec_ratios_f vec_ratios_b;

In [ ]:
f = Figure(size = (size_pt[1]*1.25, size_pt[2]*0.9), figure_padding = 1)
ga = f[1,1] = GridLayout()

# Median ± interquartile ranges
xi = 0.0; xf = 2.1
ax1 = Axis(ga[1,1], xticks=(0:1:xf, ["0", "1", "2"]), yticks=(0:1:xf, ["0", "1", "2"]), 
           xlabel="True ratio", ylabel="Observed ratio")

x = ratio_f_th_dep[inds]
y = median.(vec_ratios_f)
y_err_Q1 = y  .- quantile.(vec_ratios_f, 0.25)
y_err_Q3 = quantile.(vec_ratios_f, 0.75) .- y
errorbars!(ax1, x, y, y_err_Q1, y_err_Q3, whiskerwidth=3, linewidth=0.6, color=(c2, 0.2), direction=:y)
scatter!(ax1, x, y, markersize=3, color=(c2, 0.5))

lines!(ax1, 0:0.001:xf, 0:0.001:xf, color=(:black, 0.4), linestyle=:dash, linewidth=0.5)
xlims!(ax1, xi, xf)
ylims!(ax1, xi, xf)

# Median ± interquartile ranges

xi = 0.0; xf = 2.5
ax2 = Axis(ga[1,2], xticks=(0:1.0:xf), yticks=(0:1.0:xf))

x = ratio_b_th_dep[inds]
y = median.(vec_ratios_b)
y_err_Q1 = y  .- quantile.(vec_ratios_b, 0.25)
y_err_Q3 = quantile.(vec_ratios_b, 0.75) .- y
errorbars!(ax2, x, y, y_err_Q1, y_err_Q3, whiskerwidth=3, linewidth=0.6, color=(c1, 0.2), direction=:y)
scatter!(ax2, x, y, markersize=3, color=(c1, 0.5))

lines!(ax2, 0:0.001:xf, 0:0.001:xf, color=(:black, 0.4), linestyle=:dash, linewidth=0.5)
xlims!(ax2, xi, xf)
ylims!(ax2, xi, xf)

colgap!(ga, 15)

f